In [1]:
import os
import glob
import time
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

# import from py

In [2]:
import sys
sys.path.append("../../")

%run ../../ai/tf/setup/sess.py
%run ../../ai/tf/setup/inits.py
%run ../../ai/tf/setup/hooks.py

%run ../../ai/tf/ops/cells.py
%run ../../ai/tf/ops/convs.py
%run ../../ai/tf/ops/weights.py
%run ../../ai/tf/ops/regularizes.py
%run ../../ai/dataset/everyone/tfrecord/tfreader.py

In [3]:
%run ../../ai/model/exp/frame/blocks.py

# load dataset

In [4]:
tf_src_path = "/home/chy/archive-data/processed/everyone-tfr"
tfrecord_files = glob.glob(os.path.join(tf_src_path, '*.tfr'))

for_train = tfrecord_files[0:150]
for_val = tfrecord_files[150:185]

batch_size = 8
iter_train = to_dataset(for_train, batch_size=batch_size)
frames, lefts, rights, cam_x, cam_y, csx, csy = iter_train.get_next()

iter_val = to_dataset(for_val, batch_size=8)
frames_v, _, _, cam_x_v, cam_y_v, csx_v, csy_v = iter_val.get_next()

In [5]:
train_item = len(for_train) * 16 * 512
step_per_epoch = int(train_item / batch_size)

# build model

In [6]:
# TODO: ModelInputs 클래스에서 purpose에 따른 동적 dispatch
pred_x, pred_y = build_model(frames, csx, csy, True)

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


# loss

In [7]:
labels = tf.transpose(tf.stack([cam_x, cam_y]))
preds = tf.transpose(tf.stack([pred_x, pred_y]))
mse = tf.losses.mean_squared_error(labels, preds)

loss_x = tf.losses.mean_squared_error(cam_x, pred_x)
loss_y = tf.losses.mean_squared_error(cam_y, pred_y)
loss = loss_x + loss_y

# optimizer

In [8]:
lr_init = 0.001        
decay_steps = 2500
lr_decay_factor = 0.8  

lr = tf.train.exponential_decay(
        learning_rate=lr_init,
        global_step=tf.train.get_or_create_global_step(),
        decay_steps=int(decay_steps),
        decay_rate=lr_decay_factor,
        staircase=True)
    
train_op = tf.train.AdamOptimizer(lr).minimize(loss)

In [9]:
global_step = tf.train.get_or_create_global_step()
inc_global_step_op = tf.assign(global_step, global_step+1)
init_ops = get_init_ops(iter_train)

In [10]:
hooks = make_hooks("./")

INFO:tensorflow:Create CheckpointSaverHook.


In [11]:
with tf.Session(config=get_sess_config()) as sess:
    sess.run(init_ops)
    
    for i in range(step_per_epoch * 20):
        v_mse, v_x, v_y, _ = sess.run([mse, loss_x, loss_y, train_op])
        
        if i % 50 == 0:
            print("[{:06d}] mse:{:.2f}  ({:.2f},{:.2f})".format(i, v_mse, v_x, v_y))
        
        if i % step_per_epoch == 0:
            print("todo validate")
            
        step = sess.run(inc_global_step_op)

[000000] mse:43.72  (9.59,77.85)
todo validate
[000050] mse:33.34  (21.12,45.57)
[000100] mse:49.02  (76.66,21.38)
[000150] mse:33.71  (25.63,41.79)
[000200] mse:45.14  (36.03,54.26)
[000250] mse:6.74  (10.85,2.64)
[000300] mse:11.07  (14.18,7.97)
[000350] mse:32.14  (11.05,53.24)
[000400] mse:12.91  (12.95,12.87)
[000450] mse:11.70  (9.68,13.72)
[000500] mse:19.89  (30.20,9.57)
[000550] mse:21.64  (27.70,15.58)
[000600] mse:9.29  (13.85,4.74)
[000650] mse:14.77  (22.34,7.21)
[000700] mse:50.86  (27.59,74.13)
[000750] mse:8.74  (9.31,8.17)
[000800] mse:22.32  (24.45,20.19)
[000850] mse:25.98  (28.29,23.67)
[000900] mse:13.66  (13.73,13.59)
[000950] mse:11.44  (13.95,8.93)
[001000] mse:10.68  (8.37,13.00)
[001050] mse:16.00  (24.71,7.30)
[001100] mse:11.52  (14.53,8.51)
[001150] mse:10.42  (13.84,6.99)
[001200] mse:21.11  (11.67,30.55)
[001250] mse:25.91  (17.00,34.81)
[001300] mse:9.73  (12.64,6.82)
[001350] mse:33.06  (16.76,49.37)
[001400] mse:32.23  (23.96,40.51)
[001450] mse:5.99  

[012600] mse:5.54  (4.98,6.09)
[012650] mse:2.87  (3.48,2.27)
[012700] mse:5.55  (3.36,7.74)
[012750] mse:17.88  (24.51,11.24)
[012800] mse:4.17  (1.84,6.51)
[012850] mse:8.42  (9.11,7.72)
[012900] mse:6.83  (4.86,8.80)
[012950] mse:14.92  (8.32,21.51)
[013000] mse:4.03  (7.15,0.90)
[013050] mse:6.61  (9.17,4.05)
[013100] mse:2.32  (3.84,0.81)
[013150] mse:7.81  (2.88,12.74)
[013200] mse:6.23  (6.57,5.90)
[013250] mse:4.64  (2.64,6.63)
[013300] mse:2.86  (1.44,4.28)
[013350] mse:3.76  (2.40,5.12)
[013400] mse:6.65  (3.25,10.04)
[013450] mse:8.23  (7.87,8.58)
[013500] mse:4.13  (4.37,3.90)
[013550] mse:6.28  (3.73,8.84)
[013600] mse:13.49  (24.01,2.96)
[013650] mse:12.08  (13.87,10.29)
[013700] mse:6.84  (4.86,8.82)
[013750] mse:6.20  (4.79,7.61)
[013800] mse:5.55  (3.26,7.83)
[013850] mse:16.08  (3.72,28.43)
[013900] mse:8.56  (5.90,11.21)
[013950] mse:5.65  (3.19,8.12)
[014000] mse:7.06  (4.53,9.60)
[014050] mse:2.81  (3.16,2.47)
[014100] mse:6.05  (7.20,4.91)
[014150] mse:9.96  (16.4

[025700] mse:3.58  (5.63,1.53)
[025750] mse:3.46  (3.35,3.56)
[025800] mse:6.18  (7.11,5.26)
[025850] mse:6.58  (7.23,5.93)
[025900] mse:5.67  (7.78,3.56)
[025950] mse:9.03  (7.81,10.25)
[026000] mse:9.24  (4.84,13.65)
[026050] mse:5.99  (2.90,9.08)
[026100] mse:3.28  (2.20,4.36)
[026150] mse:6.81  (7.78,5.83)
[026200] mse:3.20  (3.87,2.52)
[026250] mse:8.00  (12.80,3.20)
[026300] mse:2.97  (1.32,4.62)
[026350] mse:5.73  (3.99,7.47)
[026400] mse:2.38  (1.79,2.96)
[026450] mse:8.77  (7.20,10.34)
[026500] mse:3.87  (2.87,4.88)
[026550] mse:3.29  (3.28,3.29)
[026600] mse:3.12  (3.57,2.68)
[026650] mse:6.01  (8.80,3.22)
[026700] mse:5.93  (2.27,9.59)
[026750] mse:7.86  (1.63,14.09)
[026800] mse:27.66  (18.43,36.90)
[026850] mse:5.00  (3.63,6.38)
[026900] mse:3.65  (3.84,3.46)
[026950] mse:10.07  (3.43,16.71)
[027000] mse:10.47  (9.52,11.42)
[027050] mse:4.31  (4.08,4.55)
[027100] mse:4.77  (3.10,6.43)
[027150] mse:2.34  (1.85,2.82)
[027200] mse:8.30  (11.70,4.90)
[027250] mse:6.32  (4.59,8

[038800] mse:5.23  (3.41,7.05)
[038850] mse:4.04  (4.14,3.93)
[038900] mse:2.74  (3.11,2.38)
[038950] mse:3.83  (4.75,2.90)
[039000] mse:2.82  (1.51,4.13)
[039050] mse:2.20  (2.21,2.18)
[039100] mse:4.37  (2.72,6.02)
[039150] mse:2.95  (3.33,2.58)
[039200] mse:6.60  (10.73,2.47)
[039250] mse:15.44  (6.43,24.44)
[039300] mse:4.04  (2.99,5.10)
[039350] mse:5.38  (5.05,5.70)
[039400] mse:5.82  (6.49,5.14)
[039450] mse:2.80  (1.57,4.03)
[039500] mse:3.12  (2.39,3.85)
[039550] mse:3.86  (5.64,2.07)
[039600] mse:3.60  (2.00,5.19)
[039650] mse:3.48  (4.80,2.16)
[039700] mse:5.78  (6.03,5.54)
[039750] mse:2.40  (2.40,2.40)
[039800] mse:8.96  (7.59,10.33)
[039850] mse:4.24  (2.14,6.34)
[039900] mse:2.70  (3.11,2.30)
[039950] mse:5.70  (6.44,4.97)
[040000] mse:4.35  (3.42,5.27)
[040050] mse:7.55  (8.74,6.36)
[040100] mse:3.72  (2.69,4.76)
[040150] mse:3.23  (4.36,2.10)
[040200] mse:7.10  (4.65,9.55)
[040250] mse:7.88  (2.91,12.85)
[040300] mse:4.49  (5.56,3.42)
[040350] mse:6.12  (4.76,7.48)
[04

[051950] mse:3.89  (3.38,4.39)
[052000] mse:3.43  (5.57,1.29)
[052050] mse:4.27  (2.48,6.07)
[052100] mse:5.05  (2.84,7.27)
[052150] mse:3.65  (3.29,4.01)
[052200] mse:3.39  (5.52,1.26)
[052250] mse:4.69  (1.64,7.75)
[052300] mse:2.70  (3.52,1.87)
[052350] mse:4.28  (1.32,7.24)
[052400] mse:2.01  (1.54,2.49)
[052450] mse:2.62  (2.71,2.53)
[052500] mse:1.76  (0.95,2.56)
[052550] mse:8.22  (2.86,13.57)
[052600] mse:5.42  (8.96,1.88)
[052650] mse:4.33  (3.05,5.60)
[052700] mse:2.55  (3.47,1.64)
[052750] mse:5.12  (6.80,3.44)
[052800] mse:2.84  (1.87,3.81)
[052850] mse:2.61  (2.47,2.75)
[052900] mse:3.73  (3.72,3.73)
[052950] mse:6.09  (3.16,9.01)
[053000] mse:2.53  (2.21,2.85)
[053050] mse:3.27  (2.81,3.74)
[053100] mse:2.05  (2.68,1.43)
[053150] mse:2.63  (2.31,2.95)
[053200] mse:8.72  (7.31,10.14)
[053250] mse:15.88  (30.90,0.86)
[053300] mse:2.21  (2.09,2.34)
[053350] mse:6.10  (3.29,8.91)
[053400] mse:3.45  (3.74,3.17)
[053450] mse:2.20  (2.09,2.31)
[053500] mse:11.75  (8.59,14.91)
[0

...

[091400] mse:7.82  (7.46,8.19)
[091450] mse:7.45  (11.98,2.91)
[091500] mse:1.64  (2.26,1.03)
[091550] mse:5.49  (6.24,4.74)
[091600] mse:7.02  (11.79,2.25)
[091650] mse:1.64  (1.70,1.58)
[091700] mse:4.17  (2.59,5.75)
[091750] mse:4.55  (4.53,4.57)
[091800] mse:7.87  (9.98,5.76)
[091850] mse:7.44  (5.80,9.09)
[091900] mse:2.45  (3.90,1.00)
[091950] mse:1.53  (1.29,1.77)
[092000] mse:2.39  (2.72,2.05)
[092050] mse:8.41  (4.79,12.03)
[092100] mse:5.42  (1.73,9.11)
[092150] mse:1.51  (0.72,2.30)
[092200] mse:2.56  (2.95,2.18)
[092250] mse:2.74  (1.38,4.10)
[092300] mse:5.02  (8.81,1.24)
[092350] mse:2.54  (1.49,3.59)
[092400] mse:3.08  (3.24,2.92)
[092450] mse:2.57  (2.35,2.79)
[092500] mse:2.65  (3.81,1.48)
[092550] mse:4.83  (1.59,8.07)
[092600] mse:3.88  (4.73,3.02)
[092650] mse:5.02  (4.08,5.96)
[092700] mse:4.20  (5.91,2.48)
[092750] mse:1.83  (2.35,1.31)
[092800] mse:4.25  (2.93,5.56)
[092850] mse:2.36  (2.03,2.70)
[092900] mse:2.29  (1.94,2.65)
[092950] mse:3.49  (3.32,3.67)
[0930

[104550] mse:6.52  (2.18,10.86)
[104600] mse:4.03  (2.24,5.82)
[104650] mse:6.11  (6.85,5.37)
[104700] mse:2.69  (3.26,2.11)
[104750] mse:4.26  (6.72,1.80)
[104800] mse:5.01  (2.42,7.60)
[104850] mse:2.19  (1.88,2.50)
[104900] mse:4.53  (2.65,6.42)
[104950] mse:2.20  (2.43,1.96)
[105000] mse:3.75  (2.53,4.96)
[105050] mse:4.74  (3.57,5.91)
[105100] mse:6.48  (10.91,2.06)
[105150] mse:10.65  (9.99,11.30)
[105200] mse:6.50  (0.94,12.07)
[105250] mse:9.49  (10.72,8.27)
[105300] mse:1.10  (1.42,0.79)
[105350] mse:3.85  (3.07,4.62)
[105400] mse:8.89  (5.33,12.45)
[105450] mse:3.21  (5.77,0.65)
[105500] mse:6.24  (5.99,6.48)
[105550] mse:1.79  (2.67,0.91)
[105600] mse:4.56  (5.21,3.91)
[105650] mse:3.25  (3.33,3.17)
[105700] mse:1.15  (1.37,0.92)
[105750] mse:4.88  (5.59,4.16)
[105800] mse:1.46  (1.06,1.86)
[105850] mse:3.50  (2.29,4.71)
[105900] mse:1.50  (0.98,2.01)
[105950] mse:10.07  (18.23,1.91)
[106000] mse:4.21  (5.97,2.45)
[106050] mse:2.58  (2.02,3.14)
[106100] mse:2.87  (3.66,2.08)

[117650] mse:3.68  (3.88,3.48)
[117700] mse:2.42  (3.13,1.70)
[117750] mse:2.69  (2.52,2.86)
[117800] mse:10.66  (5.62,15.70)
[117850] mse:8.89  (9.02,8.75)
[117900] mse:4.75  (2.21,7.29)
[117950] mse:2.63  (0.68,4.57)
[118000] mse:2.69  (3.72,1.67)
[118050] mse:7.34  (9.71,4.98)
[118100] mse:3.76  (0.49,7.03)
[118150] mse:1.64  (2.37,0.91)
[118200] mse:2.06  (2.14,1.99)
[118250] mse:3.69  (4.21,3.16)
[118300] mse:4.20  (1.87,6.53)
[118350] mse:3.92  (3.68,4.15)
[118400] mse:1.95  (2.45,1.45)
[118450] mse:5.66  (7.00,4.32)
[118500] mse:3.61  (1.95,5.27)
[118550] mse:3.37  (4.76,1.97)
[118600] mse:3.60  (2.54,4.65)
[118650] mse:3.04  (2.97,3.10)
[118700] mse:1.99  (1.31,2.67)
[118750] mse:11.96  (14.97,8.96)
[118800] mse:1.76  (1.30,2.22)
[118850] mse:3.29  (3.58,3.00)
[118900] mse:1.80  (2.16,1.44)
[118950] mse:3.46  (2.09,4.84)
[119000] mse:1.55  (0.64,2.46)
[119050] mse:4.59  (1.93,7.25)
[119100] mse:5.79  (4.60,6.97)
[119150] mse:1.33  (1.27,1.39)
[119200] mse:5.55  (4.92,6.19)
[119

[130800] mse:5.76  (3.51,8.02)
[130850] mse:2.32  (3.42,1.22)
[130900] mse:12.31  (3.89,20.73)
[130950] mse:2.83  (2.79,2.87)
[131000] mse:2.35  (1.45,3.25)
[131050] mse:5.78  (7.06,4.51)
[131100] mse:2.92  (2.57,3.27)
[131150] mse:4.00  (5.90,2.09)
[131200] mse:2.69  (3.12,2.26)
[131250] mse:5.16  (3.50,6.83)
[131300] mse:5.89  (2.15,9.63)
[131350] mse:4.38  (5.32,3.44)
[131400] mse:1.33  (1.67,1.00)
[131450] mse:6.61  (2.77,10.45)
[131500] mse:2.18  (3.08,1.28)
[131550] mse:6.90  (12.52,1.28)
[131600] mse:1.85  (1.99,1.71)
[131650] mse:4.81  (5.18,4.44)
[131700] mse:2.90  (3.16,2.64)
[131750] mse:2.54  (1.05,4.02)
[131800] mse:3.55  (2.51,4.59)
[131850] mse:3.12  (3.52,2.73)
[131900] mse:6.80  (5.85,7.74)
[131950] mse:1.82  (1.60,2.05)
[132000] mse:1.40  (1.22,1.58)
[132050] mse:4.49  (6.82,2.16)
[132100] mse:3.81  (1.81,5.82)
[132150] mse:5.07  (0.89,9.25)
[132200] mse:7.11  (12.63,1.60)
[132250] mse:4.64  (3.33,5.94)
[132300] mse:3.99  (3.51,4.48)
[132350] mse:2.20  (2.56,1.85)
[13

KeyboardInterrupt: 

# relay learning

In [ ]:
with tf.Session(config=get_sess_config()) as sess:
    for i in range(step_per_epoch * 5):
        v_mse, v_x, v_y, _ = sess.run([mse, loss_x, loss_y, train_op])
        
        if i % 50 == 0:
            print("[{:06d}] mse:{:.2f}  ({:.2f},{:.2f})".format(i, v_mse, v_x, v_y))
        
        if i % step_per_epoch == 0:
            v_mse, v_x, v_y, _ = sess.run([mse, loss_x, loss_y, train_op])
            
        step = sess.run(inc_global_step_op)

# TODO
<br>

## TOP priority
<br >

#### everyone
- #item : 1490944
- train step in epoch : 153600 

<br>

### accuracy 
- resize 방식 변경
   - max(h, w)를 기준으로 resize하여 남는 여백을 padding으로 채우는  resize
- ~~데이터셋 iterator val, train 분리 작성~~
- epoch 별 validation 돌리기
- learning rate annealing
- gap 대신 fc 사용
- 필터수 조절
- eye dual channel
- orientation 통제해서 실험
- 카메라로부터 r,Θ 좌표로... 
- frame 비율을 줄여보면서 중요 요소 비중 실험 
- ~~eye에 대한 conv path를 2개 써서 x, y를 분리~~
- ~~마지막 레이어에서 cm 보정 (카메라와 스크린 첫 픽셀)~~
- ~~regularizer, drop out~~
- ~~dataset 개수 카운팅~~

<br>

### opts
- epoch, train time, lr
- check point save

<br>

### th
- loss가 큰 것들의 분포는 어떻게 될까?
   - 
